In [3]:
pwd

'C:\\Users\\Edgard_Cuadra\\OneDrive\\Desktop\\Ironhack\\Projects\\Project_2_Pipeline'

In [4]:
import src.clean as cl

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re 
import numpy as np
import os.path

### Extracting Experts players rankings from CBS sports 

First we find the webpage we want to get the information from so that we can scrape the data out of it. to make it legible we transform in into html format by using beutifulsoup.

In [6]:
urlcbs = "https://www.cbssports.com/fantasy/football/rankings/"

In [7]:
cbshtml = cl.get_html(urlcbs)

once we get the content in a format we can use we can proceedd to extract the information wanted, in this cases we need player name, team, position and ranking.

In [8]:
player1 = cl.select_html(cbshtml,"span.player-name")

In [9]:
player1_list = [element.getText().strip() for element in player1]

In [10]:
ranking1 = cl.select_html(cbshtml,"div.rank")

In [11]:
ranking1_list = [element.getText().strip() for element in ranking1]

In [12]:
team1 = cl.select_html(cbshtml,"span.team.position")

In [13]:
team1_list = [element.getText().strip() for element in team1]

In [14]:
Team1_ = [element.split("\n")[0].strip(",").strip()  for element in team1_list]

In [15]:
position1_ = [element.split("\n")[1].strip(",").strip()  for element in team1_list]

Now that we have extracted the information into separate lists we need to converge them into a dictionary in order for us to be able to turn it into a dataframe.

In [16]:
allRanking ={
    "Player": player1_list,
    "Position": position1_,
    "Team": Team1_,
    "Ranking" : ranking1_list,
}

In [17]:
allrankings = pd.DataFrame(allRanking)
allrankings.head(2)

,Player,Position,Team,Ranking
0,J. Taylor,RB,IND,1
1,C. McCaffrey,RB,CAR,2


the web page pulled all the rankings into one dataframe, we need proceed to great 3 dataframes with the rankings. using iloc we are going to cut the dataframe into 3 tables and after merge them with the name index.

In [18]:
eisenbergranking = allrankings.iloc[0:200,:].reset_index(drop=True)
richardranking = allrankings.iloc[200:400,:].reset_index(drop=True)
cummingsranking = allrankings.iloc[400:600,:].reset_index(drop=True)

In [19]:
from functools import reduce

In [20]:
x = [eisenbergranking, richardranking, cummingsranking]

test = reduce(lambda left,right: pd.merge(left,
right,on=['Player',"Position","Team"],how='outer'), x)

In [21]:
test.sample(3)

,Player,Position,Team,Ranking_x,Ranking_y,Ranking
31,D. Montgomery,RB,CHI,32,25,16
217,C. Wentz,QB,WAS,NaN,NaN,188
41,T. McLaurin,WR,WAS,42,45,41


since we want the top players to look out after, we need to drop the rows bellow line 100 and use the information of the top ranked prospects in the league.

In [22]:
top100ranked = test.head(120)

we check if there is any missing values of information of the collumns that we need to make sure we have the required information for all the players in the top playes, and since we have to make a calculated column we need to change the type of some columns to integers

In [23]:
cl.columnnullcount(top100ranked)

Player       0
Position     0
Team         0
Ranking_x    0
Ranking_y    0
Ranking      0
dtype: int64

In [24]:
top100ranked["Ranking_x"] = top100ranked["Ranking_x"].apply(int)

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_16832\2618516453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked["Ranking_x"] = top100ranked["Ranking_x"].apply(int)


In [25]:
top100ranked["Ranking_y"] = top100ranked["Ranking_y"].apply(int)

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_16832\230294182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked["Ranking_y"] = top100ranked["Ranking_y"].apply(int)


In [26]:
top100ranked["Ranking"] = top100ranked["Ranking"].apply(int)

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_16832\1780496477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked["Ranking"] = top100ranked["Ranking"].apply(int)


In [27]:
top100ranked.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Player     120 non-null    object
 1   Position   120 non-null    object
 2   Team       120 non-null    object
 3   Ranking_x  120 non-null    int64 
 4   Ranking_y  120 non-null    int64 
 5   Ranking    120 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 6.6+ KB


now we need to creat a new column with an average agregate of the ranking for every player to get our ranking value as an average of all three experts rankings

In [28]:
top100ranked['My_Ranking'] = (top100ranked.Ranking_x + top100ranked.Ranking_y + top100ranked.Ranking)/3

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_16832\748878491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked['My_Ranking'] = (top100ranked.Ranking_x + top100ranked.Ranking_y + top100ranked.Ranking)/3


In [29]:
top100ranked.head(2)

,Player,Position,Team,Ranking_x,Ranking_y,Ranking,My_Ranking
0,J. Taylor,RB,IND,1,1,1,1.000000
1,C. McCaffrey,RB,CAR,2,5,3,3.333333


now that we have the desired outcome i need to change the columns name, and eliminate columns that are not usefull or repetitive. after that we can rearrange by the column of my ranking and reset the index with that order.

In [30]:
top100ranked = top100ranked.rename(columns={"Ranking_x": "Eisenberg_ranking","Ranking_y": "Richards_ranking", "Ranking": "Cummings_ranking"})

In [31]:
top100ranked = top100ranked.round(2) #this will roun to 2 decimal places

In [32]:
top100ranked.sample(3)

,Player,Position,Team,Eisenberg_ranking,Richards_ranking,Cummings_ranking,My_Ranking
9,J. Chase,WR,CIN,10,10,10,10.00
96,R. Jones,RB,KC,97,94,111,100.67
55,D. Mooney,WR,CHI,56,59,45,53.33


In [33]:
top100ranked = top100ranked.sort_values(by=["My_Ranking"]).reset_index(drop=True)

In [34]:
top100ranked.sample(3)

,Player,Position,Team,Eisenberg_ranking,Richards_ranking,Cummings_ranking,My_Ranking
3,D. Henry,RB,TEN,5,2,6,4.33
46,M. Williams,WR,LAC,46,50,44,46.67
13,D. Adams,WR,LV,16,11,15,14.00


### Importing players stats and salary.

the database that is being impor is the general starts of all the NFL players and their official salary. 

In [35]:
pwd

'C:\\Users\\Edgard_Cuadra\\OneDrive\\Desktop\\Ironhack\\Projects\\Project_2_Pipeline'

In [36]:
nfl_stat = pd.read_csv('.\\Input\\madden21_ratings.csv', encoding='latin1')

the table we imported has too much information that we dont need so we will drop the columns that are of no use to us, we can use the following code to eliminate by column position and make it easier and faster for us to drop the columns.

In [37]:
nfl_stat2 = nfl_stat.drop(nfl_stat.columns[[11,12,13,14,15,16,17,20,21,22,23,24,25,26,28,29,30,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,58,62,63]], axis=1) 

In [38]:
nfl_stat2.head(3)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Tackle,Injury,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College
0,Rams,Aaron Donald,99,RE,29,82,90,99,86,99,...,92,98,99,"$101,892,000.00","$40,000,000.00",6,73,280,5/23/1991,Pittsburgh
1,Patriots,Stephon Gilmore,99,CB,29,92,94,99,94,70,...,68,94,24,"$33,550,000.00","$31,450,000.00",8,73,202,9/19/1990,South Carolina
2,Panthers,Christian McCaffrey,99,HB,24,92,93,97,97,72,...,36,96,22,"$45,840,000.00","$32,190,000.00",3,71,205,6/7/1996,Stanford


in order to merge the two tables by thei identifying names we will have to use the same numenclatures, thus the best approach would be to break into two columns and rearrange to get the desired initial and last name format.

In [39]:
nfl_stat2['lastname'] = nfl_stat2['Full Name'].str.split(' ', expand=True)[1]

In [40]:
nfl_stat2['initials_'] =nfl_stat2['Full Name'].str.split('', expand=True)[1]

In [41]:
nfl_stat2['short_name'] =  nfl_stat2['initials_']+ '. ' + nfl_stat2['lastname']

now we need to start to eliminate the columns that we created temporarly to have a clean table, with only the information that we need.

In [42]:
nfl_stat2 = nfl_stat2.drop(nfl_stat2.columns[[25,26]], axis=1)

In [43]:
len(nfl_stat2.columns)

26

In [44]:
nfl_stat2.columns[25]

'short_name'

In [45]:
nfl_stat2.sample(2)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Injury,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,short_name
934,Lions,Marvin Hall Jr,71,WR,27,92,92,75,87,59,...,85,17,"$740,000.00",$-,3,70,190,4/10/1993,Washington,M. Hall
2201,Titans,Aaron Brewer,54,C,22,69,76,62,67,78,...,86,62,"$2,250,000.00","$40,000.00",0,73,274,10/28/1997,Texas State,A. Brewer


In [46]:
nfl_stat2 = nfl_stat2.rename(columns={"short_name": "Player"})

In [47]:
nfl_stat2.sample(5)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Injury,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player
1827,Buccaneers,Pat O'Connor,62,RE,26,78,88,50,71,83,...,88,79,"$660,000.00",$-,3,76,270,11/1/1993,Eastern Michigan,P. O'Connor
491,Packers,Mason Crosby,77,K,35,60,64,63,61,60,...,90,2,"$9,900,000.00","$3,000,000.00",13,73,207,9/3/1984,Colorado,M. Crosby
785,Bears,Eddy Pineiro,72,K,24,78,82,58,62,42,...,85,15,"$1,640,000.00","$20,000.00",2,71,177,9/13/1995,Florida,E. Pineiro
388,Jets,Steve McLendon,79,DT,34,61,77,84,66,86,...,90,99,"$3,010,000.00","$2,240,000.00",11,75,310,1/3/1986,Troy,S. McLendon
2170,Chargers,Storm Norton,55,RT,26,68,81,55,71,83,...,84,74,"$1,530,000.00",$-,3,79,317,5/16/1994,Toledo,S. Norton


in order to merge both tables we need to tie the name with the nick name so we proceed to impor another table that will help us get the missing piece of the puzzel and complete the tables before me merge them.

In [48]:
teamlist = pd.read_csv('.\\Input\\Teamslist.csv', encoding='latin1')

In [49]:
p = [nfl_stat2, teamlist]

final = reduce(lambda left,right: pd.merge(left,
right,on=['Team'],how='outer'), p)

In [50]:
final.head(2)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,ï»¿city
0,Rams,Aaron Donald,99,RE,29,82,90,99,86,99,...,99,"$101,892,000.00","$40,000,000.00",6,73,280,5/23/1991,Pittsburgh,A. Donald,LAR
1,Rams,Jalen Ramsey,94,CB,25,91,93,94,90,69,...,20,"$18,730,000.00","$13,250,000.00",4,73,208,10/24/1994,Florida State,J. Ramsey,LAR


Now that we have the table renamed we can proceed to change some colum names so they match when mergin.

In [51]:
final = final.rename(columns={"Team": "Nickname"})

In [52]:
final = final.rename(columns={"ï»¿city": "Team"})

In [53]:
final

,Nickname,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,Team
0,Rams,Aaron Donald,99,RE,29,82,90,99,86,99,...,99,"$101,892,000.00","$40,000,000.00",6,73,280,5/23/1991,Pittsburgh,A. Donald,LAR
1,Rams,Jalen Ramsey,94,CB,25,91,93,94,90,69,...,20,"$18,730,000.00","$13,250,000.00",4,73,208,10/24/1994,Florida State,J. Ramsey,LAR
2,Rams,Cooper Kupp,89,WR,27,88,91,93,95,65,...,10,"$4,070,000.00","$950,000.00",3,74,208,6/15/1993,Eastern Wash.,C. Kupp,LAR
3,Rams,Robert Woods,86,WR,28,90,88,90,87,63,...,17,"$31,000,000.00","$8,020,000.00",7,72,193,4/10/1992,USC,R. Woods,LAR
4,Rams,Johnny Hekker,86,P,30,69,79,72,64,56,...,6,"$16,560,000.00","$4,710,000.00",8,77,241,2/8/1990,Oregon State,J. Hekker,LAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,Lions,Beau Benzschawel,58,C,24,64,77,69,68,82,...,63,"$1,760,000.00",$-,1,78,300,9/10/1995,Wisconsin,B. Benzschawel,DET
2289,Lions,John Atkins,58,DT,27,60,73,55,54,81,...,99,"$590,000.00",$-,2,75,320,12/21/1992,Georgia,J. Atkins,DET
2290,Lions,David Blough,56,QB,25,75,86,64,76,58,...,10,"$1,760,000.00",$-,1,72,200,7/31/1995,Purdue,D. Blough,DET
2291,Lions,Dan Skipper,54,RT,25,58,75,54,64,81,...,70,"$670,000.00",$-,3,81,325,9/20/1994,Arkansas,D. Skipper,DET


Once the data base is clean and ready to merge with our top players df , then we proceed to make sure some names from our top ranks are callable. this will give us the ability to check if random names from a table can be found in the other befor we merge.

In [54]:
query = final [(final["Player"] =="D. Smith")]

In [55]:
query

,Nickname,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,Team
530,Cowboys,Devin Smith,70,WR,28,90,91,68,88,58,...,15,"$1,390,000.00",$-,5,73,199,3/3/1992,Ohio State,D. Smith,DAL
1173,Buccaneers,Donovan Smith,70,LT,27,71,80,87,63,88,...,76,"$14,450,000.00","$26,800,000.00",5,78,338,6/23/1993,Penn State,D. Smith,TB


Now that we are certain that theres a match between this column its time to concatinate the tables, we want a inner cause we want to bring only the informarion of the top players and not all the 2500 players in the NFL league

In [56]:
topranked2 = top100ranked.set_index(['Player','Position','Team'],drop=False)

In [57]:
final2 = final.set_index(['Player','Position','Team'], drop=False)

In [58]:
final2. sample(5)

,,,Nickname,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,Team
Player,Position,Team,,,,,,,,,,,,,,,,,,,,,
T. Williams,HB,CIN,Bengals,Trayveon Williams,65,HB,22,89,90,68,83,64,...,32,"$2,540,000.00","$160,000.00",1,68,206,10/18/1997,Texas AM,T. Williams,CIN
J. Moore,WR,CLE,Browns,J'Mon Moore,66,WR,25,87,88,53,94,65,...,16,"$1,270,000.00",$-,3,75,205,5/23/1995,Missouri,J. Moore,CLE
M. LaCosse,TE,NE,Patriots,Matt LaCosse,66,TE,27,81,88,76,72,74,...,83,"$2,280,000.00","$500,000.00",5,78,255,9/21/1992,Illinois,M. LaCosse,NE
M. Brown,HB,LAR,Rams,Malcolm Brown,75,HB,27,88,91,81,85,74,...,34,"$1,090,000.00","$50,000.00",5,71,222,5/15/1993,Texas,M. Brown,LAR
R. Ortiz,FB,NO,Saints,Ricky Ortiz,64,FB,26,76,84,56,77,76,...,30,"$670,000.00",$-,3,72,233,4/15/1994,Oregon State,R. Ortiz,NO


In [59]:
final3 = final2.drop_duplicates(subset=["Player","Team"], keep=False)

In [60]:
toprank_stats = pd.concat([topranked2, final3], axis=1, join='inner')

In [61]:
toprank_stats.sample(5)

,,,Player,Position,Team,Eisenberg_ranking,Richards_ranking,Cummings_ranking,My_Ranking,Nickname,Full Name,Overall Rating,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,Team
Player,Position,Team,,,,,,,,,,,,,,,,,,,,,
D. Mooney,WR,CHI,D. Mooney,WR,CHI,56,59,45,53.33,Bears,Darnell Mooney,66,...,11,"$2,917,050.00","$280,000.00",0,70,176,10/29/1997,Tulane,D. Mooney,CHI
M. Thomas,WR,NO,M. Thomas,WR,NO,58,66,54,59.33,Saints,Michael Thomas,99,...,13,"$62,750,000.00","$35,130,000.00",4,75,212,3/3/1993,Ohio State,M. Thomas,NO
M. Andrews,TE,BAL,M. Andrews,TE,BAL,23,26,12,20.33,Ravens,Mark Andrews,86,...,89,"$2,620,000.00","$820,000.00",2,77,253,9/6/1995,Oklahoma,M. Andrews,BAL
P. Mahomes,QB,KC,P. Mahomes,QB,KC,52,47,48,49.00,Chiefs,Patrick Mahomes,99,...,15,"$6,840,000.00","$34,420,000.00",3,75,230,9/17/1995,Texas Tech,P. Mahomes,KC
M. Evans,WR,TB,M. Evans,WR,TB,18,20,24,20.67,Buccaneers,Mike Evans,92,...,13,"$49,250,000.00","$46,510,000.00",6,77,231,8/21/1993,Texas AM,M. Evans,TB


Now we check the information of every column and its stats to see general info of our dataframe. 

In [62]:
toprank_stats.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 45 entries, ('C. Kupp', 'WR', 'LAR') to ('T. Boyd', 'WR', 'CIN')
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Player             45 non-null     object 
 1   Position           45 non-null     object 
 2   Team               45 non-null     object 
 3   Eisenberg_ranking  45 non-null     int64  
 4   Richards_ranking   45 non-null     int64  
 5   Cummings_ranking   45 non-null     int64  
 6   My_Ranking         45 non-null     float64
 7   Nickname           45 non-null     object 
 8   Full Name          45 non-null     object 
 9   Overall Rating     45 non-null     int64  
 10  Position           45 non-null     object 
 11  Age                45 non-null     int64  
 12  Speed              45 non-null     int64  
 13  Acceleration       45 non-null     int64  
 14  Awareness          45 non-null     int64  
 15  Agility            45 non-null 

In [63]:
toprank_stats.describe().round(2)

,Eisenberg_ranking,Richards_ranking,Cummings_ranking,My_Ranking,Overall Rating,Age,Speed,Acceleration,Awareness,Agility,...,Stamina,Carrying,Catching,Jumping,Tackle,Injury,Jersey Number,Years Pro,Height,Weight
count,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,...,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00
mean,54.93,55.96,55.47,55.45,81.89,24.82,88.07,89.49,83.56,87.11,...,88.62,72.67,78.07,85.82,34.33,89.29,33.29,2.87,74.20,216.87
std,30.57,30.08,31.86,30.27,9.45,3.63,5.73,4.31,10.15,5.22,...,5.29,4.91,20.41,9.12,6.99,4.31,33.84,3.40,2.42,21.80
min,4.00,8.00,2.00,4.67,66.00,21.00,60.00,66.00,62.00,70.00,...,77.00,53.00,26.00,47.00,24.00,77.00,1.00,0.00,70.00,176.00
25%,28.00,34.00,30.00,29.67,75.00,23.00,86.00,88.00,75.00,84.00,...,85.00,71.00,81.00,84.00,29.00,87.00,12.00,1.00,73.00,203.00
50%,54.00,54.00,52.00,53.33,81.00,24.00,89.00,90.00,85.00,87.00,...,88.00,74.00,84.00,88.00,33.00,90.00,14.00,2.00,74.00,215.00
75%,77.00,76.00,77.00,75.67,89.00,26.00,91.00,92.00,92.00,92.00,...,91.00,76.00,90.00,91.00,37.00,91.00,81.00,4.00,76.00,230.00
max,120.00,137.00,133.00,130.00,99.00,43.00,96.00,96.00,99.00,97.00,...,99.00,80.00,99.00,99.00,55.00,99.00,89.00,20.00,78.00,260.00


Finally we have a database that is usefull to analize, now we will proceed to save as a csv file to proceed to do data vizualization on the database

In [64]:
pwd

'C:\\Users\\Edgard_Cuadra\\OneDrive\\Desktop\\Ironhack\\Projects\\Project_2_Pipeline'

In [65]:
toprank_stats.to_csv(".\\input\\toprank_stats.csv")